In [14]:
import json

tmp = json.loads("""
{"/mnt/data/radgraph/data/fake_reports/report2.txt": {"text": "Lungs are clear . No focal acute infiltrate . No pleural effusion .", "entities": {"1": {"tokens": "Lungs", "label": "ANAT-DP", "start_ix": 0, "end_ix": 0, "relations": []}, "2": {"tokens": "clear", "label": "OBS-DP", "start_ix": 2, "end_ix": 2, "relations": [["located_at", "1"]]}, "3": {"tokens": "focal", "label": "OBS-DA", "start_ix": 5, "end_ix": 5, "relations": [["modify", "5"]]}, "4": {"tokens": "acute", "label": "OBS-DA", "start_ix": 6, "end_ix": 6, "relations": [["modify", "5"]]}, "5": {"tokens": "infiltrate", "label": "OBS-DA", "start_ix": 7, "end_ix": 7, "relations": []}, "6": {"tokens": "pleural", "label": "ANAT-DP", "start_ix": 10, "end_ix": 10, "relations": []}, "7": {"tokens": "effusion", "label": "OBS-DA", "start_ix": 11, "end_ix": 11, "relations": [["located_at", "6"]]}}, "data_source": null, "data_split": "inference"}, "/mnt/data/radgraph/data/fake_reports/report.txt": {"text": "The lungs are clear , without evidence of focal acute infiltrate or effusion .", "entities": {"1": {"tokens": "lungs", "label": "ANAT-DP", "start_ix": 1, "end_ix": 1, "relations": []}, "2": {"tokens": "clear", "label": "OBS-DP", "start_ix": 3, "end_ix": 3, "relations": [["located_at", "1"]]}, "3": {"tokens": "focal", "label": "OBS-DA", "start_ix": 8, "end_ix": 8, "relations": [["modify", "5"]]}, "4": {"tokens": "acute", "label": "OBS-DA", "start_ix": 9, "end_ix": 9, "relations": [["modify", "5"]]}, "5": {"tokens": "infiltrate", "label": "OBS-DA", "start_ix": 10, "end_ix": 10, "relations": []}, "6": {"tokens": "effusion", "label": "OBS-DA", "start_ix": 12, "end_ix": 12, "relations": []}}, "data_source": null, "data_split": "inference"}}
""")

In [15]:
tmp

{'/mnt/data/radgraph/data/fake_reports/report2.txt': {'text': 'Lungs are clear . No focal acute infiltrate . No pleural effusion .',
  'entities': {'1': {'tokens': 'Lungs',
    'label': 'ANAT-DP',
    'start_ix': 0,
    'end_ix': 0,
    'relations': []},
   '2': {'tokens': 'clear',
    'label': 'OBS-DP',
    'start_ix': 2,
    'end_ix': 2,
    'relations': [['located_at', '1']]},
   '3': {'tokens': 'focal',
    'label': 'OBS-DA',
    'start_ix': 5,
    'end_ix': 5,
    'relations': [['modify', '5']]},
   '4': {'tokens': 'acute',
    'label': 'OBS-DA',
    'start_ix': 6,
    'end_ix': 6,
    'relations': [['modify', '5']]},
   '5': {'tokens': 'infiltrate',
    'label': 'OBS-DA',
    'start_ix': 7,
    'end_ix': 7,
    'relations': []},
   '6': {'tokens': 'pleural',
    'label': 'ANAT-DP',
    'start_ix': 10,
    'end_ix': 10,
    'relations': []},
   '7': {'tokens': 'effusion',
    'label': 'OBS-DA',
    'start_ix': 11,
    'end_ix': 11,
    'relations': [['located_at', '6']]}},
  'data

In [13]:
from medvqa.utils.hashing import hash_string

def compute_label_set(data):
    entities = data['entities']
    n = len(entities)
    e_strings = [None] * n
    strings = []
    hashes = set()
    for k, e in entities.items():
        i = int(k)-1
        e_strings[i] = f"{e['tokens']}|{e['label']}" # tokens|label
        hashes.add(hash_string(e_strings[i]))
        strings.append(e_strings[i])
    for k, e in entities.items():
        i = int(k)-1
        for r in e['relations']:
            j = int(r[1])-1
            rel_s1 = f"{e_strings[i]}|{r[0]}|{e_strings[j]}" # e1|rel|e2
            rel_s2 = f"{e_strings[i]}|{e_strings[j]}" # e1|e2
            hashes.add(hash_string(rel_s1))
            hashes.add(hash_string(rel_s2))
            strings.append(rel_s1)
            strings.append(rel_s2)
    return strings, hashes

In [18]:
def print_entities_and_relations(data):
    print('Original text:')
    print(data['text'])
    print()
    entities = data['entities']
    n = len(entities)
    e_pairs = [None] * n
    for k, e in entities.items():
        e_pair = (e['tokens'], e['label'])
        e_pairs[int(k)-1] = e_pair
    relations = []
    for k, e in entities.items():
        i = int(k)-1
        for r in e['relations']:
            j = int(r[1])-1
            relations.append((e_pairs[i], r[0], e_pairs[j]))
    print('Entities:')
    for x in e_pairs:
        print(x)
    print()
    print('Relations:')
    for x in relations:
        print(x)

In [16]:
data = next(iter(tmp.values()))

In [19]:
print_entities_and_relations(data)

Original text:
Lungs are clear . No focal acute infiltrate . No pleural effusion .

Entities:
('Lungs', 'ANAT-DP')
('clear', 'OBS-DP')
('focal', 'OBS-DA')
('acute', 'OBS-DA')
('infiltrate', 'OBS-DA')
('pleural', 'ANAT-DP')
('effusion', 'OBS-DA')

Relations:
(('clear', 'OBS-DP'), 'located_at', ('Lungs', 'ANAT-DP'))
(('focal', 'OBS-DA'), 'modify', ('infiltrate', 'OBS-DA'))
(('acute', 'OBS-DA'), 'modify', ('infiltrate', 'OBS-DA'))
(('effusion', 'OBS-DA'), 'located_at', ('pleural', 'ANAT-DP'))


In [20]:
compute_label_set(data)

(['Lungs|ANAT-DP',
  'clear|OBS-DP',
  'focal|OBS-DA',
  'acute|OBS-DA',
  'infiltrate|OBS-DA',
  'pleural|ANAT-DP',
  'effusion|OBS-DA',
  'clear|OBS-DP|located_at|Lungs|ANAT-DP',
  'clear|OBS-DP|Lungs|ANAT-DP',
  'focal|OBS-DA|modify|infiltrate|OBS-DA',
  'focal|OBS-DA|infiltrate|OBS-DA',
  'acute|OBS-DA|modify|infiltrate|OBS-DA',
  'acute|OBS-DA|infiltrate|OBS-DA',
  'effusion|OBS-DA|located_at|pleural|ANAT-DP',
  'effusion|OBS-DA|pleural|ANAT-DP'],
 {(12, 309747879334862366),
  (12, 855426046901773212),
  (12, 2215373754232534800),
  (13, 1614911951124434649),
  (15, 19119047849380968),
  (15, 3677406714112377304),
  (17, 2836997461612593144),
  (26, 2987285217759247654),
  (30, 194462586615084465),
  (30, 3399811303483428276),
  (31, 2673881681973443813),
  (37, 408053797035404307),
  (37, 2527974376104740564),
  (37, 3180474612184124109),
  (42, 3185582518253405045)})

In [10]:
data['text'].split()

['The',
 'lungs',
 'are',
 'clear',
 ',',
 'without',
 'evidence',
 'of',
 'focal',
 'acute',
 'infiltrate',
 'or',
 'effusion',
 '.']

In [9]:

import subprocess

# --allennlp_bin_path /home/pamessina/miniconda3/envs/dygiepp/bin/allennlp \
# --digie_package_folder /home/pamessina/dygiepp/dygie/ \

subprocess.run("""conda run -n dygiepp python3 /home/pamessina/medvqa/medvqa/scripts/radgraph/radgraph_allennlp_inference.py \
--model_path /mnt/data/radgraph/data/models/model_checkpoint/model.tar.gz \
--dygie_package_parent_folder /home/pamessina/dygiepp/ \
--data_path /mnt/data/radgraph/data/fake_reports/ \
--out_path /home/pamessina/medvqa-workspace/tmp/radgraph/results.jsonl \
--temp_folder /home/pamessina/medvqa-workspace/tmp/radgraph/""", shell=True)
# subprocess.run('bash -c "conda activate dygiepp; python3 -V"', shell=True)

Getting paths to all the reports...
Got all the paths.
Preprocessing all the reports...
1 reports done
2 reports done
Done with preprocessing.
Running the inference now... This can take a bit of time
dygie_package_parent_folder: /home/pamessina/dygiepp/
Imported dygie successfully
/home/pamessina/dygiepp/dygie/__init__.py
Running command:  PYTHONPATH=/home/pamessina/dygiepp/:$PYTHONPATH conda run -n dygiepp allennlp predict /mnt/data/radgraph/data/models/model_checkpoint/model.tar.gz /home/pamessina/medvqa-workspace/tmp/radgraph/temp_dygie_input.json             --predictor dygie             --include-package dygie             --use-dataset-reader             --output-file /home/pamessina/medvqa-workspace/tmp/radgraph/temp_dygie_output.json             --cuda-device -1             --silent
Done with inference
Inference completed.
Postprocessing output file...
Done postprocessing.
Saving results and performing final cleanup...



2023-08-05 13:46:54,785 - INFO - allennlp.common.plugins - Plugin allennlp_models available
2023-08-05 13:46:54,804 - INFO - allennlp.models.archival - loading archive file /mnt/data/radgraph/data/models/model_checkpoint/model.tar.gz
2023-08-05 13:46:54,805 - INFO - allennlp.models.archival - extracting archive file /mnt/data/radgraph/data/models/model_checkpoint/model.tar.gz to temp dir /tmp/tmp6syt6e4e
2023-08-05 13:46:57,513 - INFO - allennlp.common.params - type = from_instances
2023-08-05 13:46:57,514 - INFO - allennlp.data.vocabulary - Loading token dictionary from /tmp/tmp6syt6e4e/vocabulary.
2023-08-05 13:46:57,514 - INFO - allennlp.common.params - model.type = dygie
2023-08-05 13:46:57,515 - INFO - allennlp.common.params - model.embedder.type = basic
2023-08-05 13:46:57,515 - INFO - allennlp.common.params - model.embedder.token_embedders.bert.type = pretrained_transformer_mismatched
2023-08-05 13:46:57,515 - INFO - allennlp.common.params - model.embedder.token_embedders.bert.m

CompletedProcess(args='conda run -n dygiepp python3 /home/pamessina/medvqa/medvqa/scripts/radgraph/radgraph_allennlp_inference.py --model_path /mnt/data/radgraph/data/models/model_checkpoint/model.tar.gz --dygie_package_parent_folder /home/pamessina/dygiepp/ --data_path /mnt/data/radgraph/data/fake_reports/ --out_path /home/pamessina/medvqa-workspace/tmp/radgraph/results.jsonl --temp_folder /home/pamessina/medvqa-workspace/tmp/radgraph/', returncode=0)

In [1]:
from medvqa.utils.files import load_jsonl
import json

In [15]:
!ls -lh /home/pamessina/medvqa-workspace/tmp/radgraph/results.json

-rw-rw-r-- 1 pamessina pamessina 1.7K Aug  5 11:26 /home/pamessina/medvqa-workspace/tmp/radgraph/results.json


In [10]:
tmp = load_jsonl('/home/pamessina/medvqa-workspace/tmp/radgraph/results.jsonl')

In [11]:
tmp

[{'/mnt/data/radgraph/data/fake_reports/report2.txt': {'text': 'Lungs are clear . No focal acute infiltrate . No pleural effusion .',
   'entities': {'1': {'tokens': 'Lungs',
     'label': 'ANAT-DP',
     'start_ix': 0,
     'end_ix': 0,
     'relations': []},
    '2': {'tokens': 'clear',
     'label': 'OBS-DP',
     'start_ix': 2,
     'end_ix': 2,
     'relations': [['located_at', '1']]},
    '3': {'tokens': 'focal',
     'label': 'OBS-DA',
     'start_ix': 5,
     'end_ix': 5,
     'relations': [['modify', '5']]},
    '4': {'tokens': 'acute',
     'label': 'OBS-DA',
     'start_ix': 6,
     'end_ix': 6,
     'relations': [['modify', '5']]},
    '5': {'tokens': 'infiltrate',
     'label': 'OBS-DA',
     'start_ix': 7,
     'end_ix': 7,
     'relations': []},
    '6': {'tokens': 'pleural',
     'label': 'ANAT-DP',
     'start_ix': 10,
     'end_ix': 10,
     'relations': []},
    '7': {'tokens': 'effusion',
     'label': 'OBS-DA',
     'start_ix': 11,
     'end_ix': 11,
     'relation

In [6]:
with open('/home/pamessina/medvqa-workspace/tmp/radgraph/results.json') as f:
    x = f.readline()
    print(json.loads(x))

{'/mnt/data/radgraph/data/fake_reports/report2.txt': {'text': 'Lungs are clear . No focal acute infiltrate . No pleural effusion .', 'entities': {'1': {'tokens': 'Lungs', 'label': 'ANAT-DP', 'start_ix': 0, 'end_ix': 0, 'relations': []}, '2': {'tokens': 'clear', 'label': 'OBS-DP', 'start_ix': 2, 'end_ix': 2, 'relations': [['located_at', '1']]}, '3': {'tokens': 'focal', 'label': 'OBS-DA', 'start_ix': 5, 'end_ix': 5, 'relations': [['modify', '5']]}, '4': {'tokens': 'acute', 'label': 'OBS-DA', 'start_ix': 6, 'end_ix': 6, 'relations': [['modify', '5']]}, '5': {'tokens': 'infiltrate', 'label': 'OBS-DA', 'start_ix': 7, 'end_ix': 7, 'relations': []}, '6': {'tokens': 'pleural', 'label': 'ANAT-DP', 'start_ix': 10, 'end_ix': 10, 'relations': []}, '7': {'tokens': 'effusion', 'label': 'OBS-DA', 'start_ix': 11, 'end_ix': 11, 'relations': [['located_at', '6']]}}, 'data_source': None, 'data_split': 'inference'}, '/mnt/data/radgraph/data/fake_reports/report.txt': {'text': 'The lungs are clear , without

In [1]:
!ls /home/pamessina/dygiepp/dygie

data	     models	 __pycache__  spacy_interface  training
__init__.py  predictors  pytest.ini   tests


In [3]:
import importlib
import sys

# Add the path to the folder where 'digie' module is defined
module_path = '/home/pamessina/dygiepp'
sys.path.append(module_path)

# Now import the module
try:
    importlib.import_module('dygie')
except ModuleNotFoundError:
    print("Module 'digie' not found. Please check if the path is correct.")

In [6]:
import dygie

In [7]:
from dygie import 

<module 'dygie' from '/home/pamessina/dygiepp/dygie/__init__.py'>

In [21]:
import json

In [23]:
from medvqa.utils.files import load_json

In [26]:
train_data = load_json('/mnt/data/radgraph/data/train.json')

In [25]:
dev_data = load_json('/mnt/data/radgraph/data/dev.json')

In [27]:
test_data = load_json('/mnt/data/radgraph/data/test.json')

In [28]:
len(train_data), len(dev_data), len(test_data)

(425, 75, 100)

In [78]:
next(iter(test_data.keys()))

'p10/p10002177/s50520012.txt'

In [80]:
test_data[next(iter(test_data.keys()))]

{'text': 'FINAL REPORT INDICATION : ___ F with cough / / Cough TECHNIQUE : PA and lateral views of the chest . COMPARISON : None . FINDINGS : The lungs are clear without focal consolidation , , or edema . The cardiomediastinal silhouette is within normal limits . No acute osseous abnormalities . IMPRESSION : No acute cardiopulmonary process .',
 'labeler_1': {'entities': {'1': {'tokens': 'lungs',
    'label': 'ANAT-DP',
    'start_ix': 28,
    'end_ix': 28,
    'relations': []},
   '2': {'tokens': 'clear',
    'label': 'OBS-DP',
    'start_ix': 30,
    'end_ix': 30,
    'relations': [['located_at', '1']]},
   '3': {'tokens': 'focal',
    'label': 'OBS-DA',
    'start_ix': 32,
    'end_ix': 32,
    'relations': [['modify', '4']]},
   '4': {'tokens': 'consolidation',
    'label': 'OBS-DA',
    'start_ix': 33,
    'end_ix': 33,
    'relations': [['located_at', '1']]},
   '5': {'tokens': 'edema',
    'label': 'OBS-DA',
    'start_ix': 37,
    'end_ix': 37,
    'relations': [['located_at', 

In [79]:
print_entities_and_relations(test_data['p10/p10002177/s50520012.txt'])

Original text:
FINAL REPORT INDICATION : ___ F with cough / / Cough TECHNIQUE : PA and lateral views of the chest . COMPARISON : None . FINDINGS : The lungs are clear without focal consolidation , , or edema . The cardiomediastinal silhouette is within normal limits . No acute osseous abnormalities . IMPRESSION : No acute cardiopulmonary process .



KeyError: 'entities'

In [31]:
chexpert_graphs = load_json('/mnt/data/radgraph/data/CheXpert_graphs.json')

In [34]:
print_entities_and_relations(chexpert_graphs['train/patient00097/study1'])

Original text:
narrative : chest 2 views : 03 / 29 / 2023 history : male , 70 years old , pre-admission ancillary studies . comparison : none impression : normal heart size and pulmonary vascularity . no focal consolidation , pleural effusion , or pneumothorax . bones are unremarkable . summary : 1-no significant abnormality accession number : 9125365033 this report has been anonymized . all dates are offset from the actual dates by a fixed interval associated with the patient .

Entities:
('normal', 'OBS-DP')
('heart', 'ANAT-DP')
('size', 'ANAT-DP')
('pulmonary', 'ANAT-DP')
('vascularity', 'ANAT-DP')
('focal', 'OBS-DA')
('consolidation', 'OBS-DA')
('pleural', 'ANAT-DP')
('effusion', 'OBS-DA')
('pneumothorax', 'OBS-DA')
('bones', 'ANAT-DP')
('unremarkable', 'OBS-DP')
('significant', 'OBS-DA')
('abnormality', 'OBS-DA')
('9125365033', 'OBS-DP')
('dates', 'ANAT-DP')
('offset', 'OBS-DP')

Relations:
(('normal', 'OBS-DP'), 'located_at', ('heart', 'ANAT-DP'))
(('size', 'ANAT-DP'), 'modify', 

In [35]:
mimiccxr_graphs = load_json('/mnt/data/radgraph/data/MIMIC-CXR_graphs.json')

In [36]:
mimiccxr_keys = list(mimiccxr_graphs.keys())

In [37]:
len(mimiccxr_keys)

220763

In [40]:
mimiccxr_graphs[mimiccxr_keys[0]]

{'text': 'FINAL REPORT INDICATION : ___ year old woman with post-op hypoxemia / / any consolidation ? TECHNIQUE : Chest PA and lateral COMPARISON : Chest radiograph from ___ FINDINGS : The lung volumes are normal . No focal consolidations . The cardiomediastinal contours are normal . A small right pleural effusion . The left pleural surfaces are normal . No pneumothorax . Stable thoracolumbar scoliosis . Stable left glenohumeral arthroplasty . Stable appearance of lumbar spinal fusion hardware . The left chest port catheter terminates in the upper right atrium , unchanged . IMPRESSION : No acute cardiopulmonary process .',
 'entities': {'1': {'tokens': 'lung',
   'label': 'ANAT-DP',
   'start_ix': 31,
   'end_ix': 31,
   'relations': []},
  '2': {'tokens': 'volumes',
   'label': 'ANAT-DP',
   'start_ix': 32,
   'end_ix': 32,
   'relations': [['modify', '1']]},
  '3': {'tokens': 'normal',
   'label': 'OBS-DP',
   'start_ix': 34,
   'end_ix': 34,
   'relations': [['located_at', '1']]},
 

In [39]:
print_entities_and_relations(mimiccxr_graphs[mimiccxr_keys[0]])

Original text:
FINAL REPORT INDICATION : ___ year old woman with post-op hypoxemia / / any consolidation ? TECHNIQUE : Chest PA and lateral COMPARISON : Chest radiograph from ___ FINDINGS : The lung volumes are normal . No focal consolidations . The cardiomediastinal contours are normal . A small right pleural effusion . The left pleural surfaces are normal . No pneumothorax . Stable thoracolumbar scoliosis . Stable left glenohumeral arthroplasty . Stable appearance of lumbar spinal fusion hardware . The left chest port catheter terminates in the upper right atrium , unchanged . IMPRESSION : No acute cardiopulmonary process .

Entities:
('lung', 'ANAT-DP')
('volumes', 'ANAT-DP')
('normal', 'OBS-DP')
('focal', 'OBS-DA')
('consolidations', 'OBS-DA')
('cardiomediastinal', 'ANAT-DP')
('contours', 'ANAT-DP')
('normal', 'OBS-DP')
('small', 'OBS-DP')
('right', 'ANAT-DP')
('pleural', 'ANAT-DP')
('effusion', 'OBS-DP')
('left', 'ANAT-DP')
('pleural', 'ANAT-DP')
('surfaces', 'ANAT-DP')
('normal',

In [42]:
from nltk.tokenize import sent_tokenize

In [43]:
sent_tokenize(mimiccxr_graphs[mimiccxr_keys[0]]['text'])

['FINAL REPORT INDICATION : ___ year old woman with post-op hypoxemia / / any consolidation ?',
 'TECHNIQUE : Chest PA and lateral COMPARISON : Chest radiograph from ___ FINDINGS : The lung volumes are normal .',
 'No focal consolidations .',
 'The cardiomediastinal contours are normal .',
 'A small right pleural effusion .',
 'The left pleural surfaces are normal .',
 'No pneumothorax .',
 'Stable thoracolumbar scoliosis .',
 'Stable left glenohumeral arthroplasty .',
 'Stable appearance of lumbar spinal fusion hardware .',
 'The left chest port catheter terminates in the upper right atrium , unchanged .',
 'IMPRESSION : No acute cardiopulmonary process .']

In [61]:
from nltk.tokenize import sent_tokenize

def _token_idx_to_sentence_idx(token_idx, sentence_offsets):
    for i in range(len(sentence_offsets)):
        if token_idx < sentence_offsets[i]:
            return i-1
    return len(sentence_offsets) - 1

def print_entities_and_relations_per_sentence(data):
    sentences = sent_tokenize(data['text'])
    ns = len(sentences)
    sentence_tokens = [s.split() for s in sentences]
    sentence_offsets = [0] * ns
    for i in range(1, ns):
        sentence_offsets[i] = sentence_offsets[i-1] + len(sentence_tokens[i-1])

    pairs_to_ent_rels = {}
    
    # Add entities
    entities = data['entities']
    n = len(entities)
    e_pairs = [None] * n
    for k, e in entities.items():
        e_idx = int(k)-1
        start_sent_idx = _token_idx_to_sentence_idx(e['start_ix'], sentence_offsets)
        end_sent_idx = _token_idx_to_sentence_idx(e['end_ix'], sentence_offsets)
        p = (start_sent_idx, end_sent_idx)
        e_string = f'{e["tokens"].lower()}|{e["label"]}'
        assert start_sent_idx <= end_sent_idx
        try:
            pairs_to_ent_rels[p].append(e_string)
        except KeyError:
            pairs_to_ent_rels[p] = [e_string]
        e_pairs[e_idx] = (e_string, p)
    
    # Add relations
    for k, e in entities.items():
        i = int(k)-1
        for r in e['relations']:
            j = int(r[1])-1
            start_sent_idx = min(e_pairs[i][1][0], e_pairs[j][1][0])
            end_sent_idx = max(e_pairs[i][1][1], e_pairs[j][1][1])
            p = (start_sent_idx, end_sent_idx)
            r_string_1 = f'{e_pairs[i][0]}|{r[0]}|{e_pairs[j][0]}' # e1|rel|e2
            r_string_2 = f'{e_pairs[i][0]}|{e_pairs[j][0]}' # e1|e2
            try:
                pairs_to_ent_rels[p].append(r_string_1)
                pairs_to_ent_rels[p].append(r_string_2)
            except KeyError:
                pairs_to_ent_rels[p] = [r_string_1, r_string_2]

    # Sort pairs by length
    pairs = list(pairs_to_ent_rels.keys())
    if len(pairs) > 1:
        pairs.sort(key=lambda x: x[1]-x[0], reverse=True)
        len_last = pairs[-1][1] - pairs[-1][0]
        for i in range(len(pairs)):
            len_i  = pairs[i][1] - pairs[i][0]
            if len_i == len_last:
                break
            for j in range(i+1, len(pairs)):
                len_j = pairs[j][1] - pairs[j][0]
                if len_i > len_j and pairs[i][0] <= pairs[j][0] and pairs[i][1] >= pairs[j][1]:
                    # pairs[i] contains pairs[j] -> include all entities and relations
                    # from pairs[j] in pairs[i]
                    pairs_to_ent_rels[pairs[i]].extend(pairs_to_ent_rels[pairs[j]])
                    del pairs_to_ent_rels[pairs[j]]
    
    # Print entities and relations
    for p, ent_rels in pairs_to_ent_rels.items():
        print('-'*80)
        print(f'[{p[0]}-{p[1]}]')
        for i in range(p[0], p[1]+1):
            print(f'\t{sentences[i]}')
        print('Entities and relations:')
        for er in ent_rels:
            print(f'\t{er}')

In [62]:
print(mimiccxr_graphs[mimiccxr_keys[-1]]['text'])
print_entities_and_relations_per_sentence(mimiccxr_graphs[mimiccxr_keys[-1]])

FINAL REPORT EXAMINATION : CHEST ( PA AND LAT ) INDICATION : ___ year old woman s / p roux-en-y bypass with temp elevation / / r / o pna COMPARISON : No comparison IMPRESSION : The lung volumes are normal . Normal size of the cardiac silhouette . Normal hilar and mediastinal contours . No pneumonia , no pulmonary edema . No pleural effusions .
--------------------------------------------------------------------------------
[0-0]
	FINAL REPORT EXAMINATION : CHEST ( PA AND LAT ) INDICATION : ___ year old woman s / p roux-en-y bypass with temp elevation / / r / o pna COMPARISON : No comparison IMPRESSION : The lung volumes are normal .
Entities and relations:
	lung|ANAT-DP
	volumes|ANAT-DP
	normal|OBS-DP
	volumes|ANAT-DP|modify|lung|ANAT-DP
	volumes|ANAT-DP|lung|ANAT-DP
	normal|OBS-DP|located_at|lung|ANAT-DP
	normal|OBS-DP|lung|ANAT-DP
--------------------------------------------------------------------------------
[1-1]
	Normal size of the cardiac silhouette .
Entities and relations:
	no

In [59]:
chexpert_keys = list(chexpert_graphs.keys())

In [60]:
print(chexpert_graphs[chexpert_keys[-1]]['text'])
print_entities_and_relations_per_sentence(chexpert_graphs[chexpert_keys[-1]])

narrative : exam : chest 1 view , 16 / 05 / 07 . history : 48 years female , neutropenic fever . comparison : 05 / 03 / 2016 . impression : 1 . slightly lower volume relative to the prior study , but compared to the prior exam , there is an apparent new small opacity in the right upper lung zone; recommend a follow-up pa and lateral exam , with a lordotic view , to evaluate for a possible new infection . 2 . no evidence of a pleural effusion . 3 . cardiac silhouette and vascularity are within normal limits . 4 . central venous catheter in appropriate position . summary : 2-abnormal , previously reported i have personally reviewed the images for this examination and agreed with the report transcribed above . accession number : 360-908-690-0 this report has been anonymized . all dates are offset from the actual dates by a fixed interval associated with the patient .
--------------------------------------------------------------------------------
[4-4]
	slightly lower volume relative to t

In [218]:
from importlib import reload
import medvqa

In [219]:
reload(medvqa.datasets.radgraph)

<module 'medvqa.datasets.radgraph' from '/home/pamessina/medvqa/medvqa/datasets/radgraph/__init__.py'>

In [220]:
from medvqa.datasets.radgraph import print_entities_and_relations_per_sentence

In [221]:
print(dev_data['p10/p10003412/s59172281.txt'])
print()
print_entities_and_relations_per_sentence(dev_data['p10/p10003412/s59172281.txt'])

{'text': 'FINAL REPORT EXAMINATION : Chest : Frontal and lateral views INDICATION : History : ___ M with dyspnea / / infiltrate TECHNIQUE : Chest : Frontal and Lateral COMPARISON : None . FINDINGS : The lungs are clear without focal consolidation . No pleural effusion or pneumothorax is seen . The cardiac and mediastinal silhouettes are unremarkable . Partially imaged hardware in the lumbar spine . IMPRESSION : No acute cardiopulmonary process .', 'entities': {'1': {'tokens': 'lungs', 'label': 'ANAT-DP', 'start_ix': 35, 'end_ix': 35, 'relations': []}, '2': {'tokens': 'clear', 'label': 'OBS-DP', 'start_ix': 37, 'end_ix': 37, 'relations': [['located_at', '1']]}, '3': {'tokens': 'focal', 'label': 'OBS-DA', 'start_ix': 39, 'end_ix': 39, 'relations': [['modify', '4']]}, '4': {'tokens': 'consolidation', 'label': 'OBS-DA', 'start_ix': 40, 'end_ix': 40, 'relations': [['located_at', '1']]}, '5': {'tokens': 'pleural', 'label': 'ANAT-DP', 'start_ix': 43, 'end_ix': 43, 'relations': []}, '6': {'tok

In [222]:
!python ../../scripts/radgraph/precompute_label_sets_per_sentence.py

Loading graphs...
Loaded train graphs from /mnt/data/radgraph/data/train.json
Loaded dev graphs from /mnt/data/radgraph/data/dev.json
Loaded test graphs from /mnt/data/radgraph/data/test.json
Loaded mimiccxr graphs from /mnt/data/radgraph/data/MIMIC-CXR_graphs.json
Loaded chexpert graphs from /mnt/data/radgraph/data/CheXpert_graphs.json
len(train_graphs) = 425
len(dev_graphs) = 75
len(test_graphs) = 100
len(mimiccxr_graphs) = 220763
len(chexpert_graphs) = 500
Computing label sets for train...
100%|████████████████████████████████████████| 425/425 [00:01<00:00, 410.37it/s]
len(hash2string) = 7736
len(output[train]) = 2170
Computing label sets for dev...
100%|██████████████████████████████████████████| 75/75 [00:00<00:00, 412.42it/s]
len(hash2string) = 8667
len(output[dev]) = 430
Computing label sets for test...
100%|████████████████████████████████████████| 100/100 [00:00<00:00, 218.95it/s]
len(hash2string) = 10168
len(output[test]) = 526
Computing label sets for mimiccxr...
100%|██████

In [223]:
!ls -lh /mnt/workspace/pamessina/medvqa-workspace/cache/radgraph/label_sets_per_sentence.pkl

-rw-rw-r-- 1 pamessina socvis 192M Aug 13 20:19 /mnt/workspace/pamessina/medvqa-workspace/cache/radgraph/label_sets_per_sentence.pkl


In [224]:
from medvqa.utils.files import load_pickle

In [225]:
tmp = load_pickle('/mnt/workspace/pamessina/medvqa-workspace/cache/radgraph/label_sets_per_sentence.pkl')

In [226]:
tmp.keys()

dict_keys(['hash2string', 'train', 'dev', 'test', 'mimiccxr', 'chexpert'])

In [227]:
for k in tmp.keys():
    print(k, len(tmp[k]))

hash2string 283682
train 2170
dev 430
test 526
mimiccxr 635245
chexpert 2270


In [228]:
strings = set()
for k in ('train', 'dev', 'test', 'chexpert', 'mimiccxr'):
# for k in ('train', 'dev', 'test'):
    strings.update(tmp[k])

In [229]:
strings_list = list(strings)
strings_list.sort(key=lambda x:(len(x), x))

In [173]:
from nltk.tokenize import sent_tokenize

In [243]:
strings_list[200]

'Heart size'

In [247]:
for k in ('train', 'dev', 'test', 'chexpert', 'mimiccxr'):
    try:
        s = strings_list[-20000]
        l = tmp[k][s]
        print(k)
        print()
        print(s)
        print()
        for x in l:
            print(tmp['hash2string'][x])
    except KeyError:
        pass

mimiccxr

IMPRESSION : Patchy opacities at the left lung base and in the right mid lung do not appear to be significantly changed , and are consistent with multifocal pneumonia

left|ANAT-DP|lung|ANAT-DP
opacities|OBS-DP|pneumonia|OBS-DP
patchy|OBS-DP|modify|opacities|OBS-DP
significantly|OBS-DA
base|ANAT-DP|lung|ANAT-DP
multifocal|OBS-DP
mid|ANAT-DP
right|ANAT-DP|modify|lung|ANAT-DP
lung|ANAT-DP
opacities|OBS-DP|located_at|lung|ANAT-DP
right|ANAT-DP
mid|ANAT-DP|lung|ANAT-DP
base|ANAT-DP|modify|lung|ANAT-DP
right|ANAT-DP|lung|ANAT-DP
opacities|OBS-DP|suggestive_of|pneumonia|OBS-DP
changed|OBS-DA|modify|opacities|OBS-DP
multifocal|OBS-DP|modify|pneumonia|OBS-DP
opacities|OBS-DP|lung|ANAT-DP
left|ANAT-DP
opacities|OBS-DP
base|ANAT-DP
patchy|OBS-DP
left|ANAT-DP|modify|lung|ANAT-DP
pneumonia|OBS-DP
changed|OBS-DA
mid|ANAT-DP|modify|lung|ANAT-DP
changed|OBS-DA|opacities|OBS-DP
patchy|OBS-DP|opacities|OBS-DP
multifocal|OBS-DP|pneumonia|OBS-DP


In [136]:
for x in mimiccxr_graphs.values():
    text = x['text']
    entities = x['entities']
    stop = False
    for y in entities.values():
        y_start = y['start_ix']
        y_end = y['end_ix']
        assert y_start <= y_end
        for r in y['relations']:
            
        if y['start_ix'] < y['end_ix']:
            stop = True
            print(text)
            print()
            print(y)
            print()
            print(entities)
            break
    if stop:
        break

FINAL REPORT EXAMINATION : CHEST ( PORTABLE AP ) INDICATION : ___ year old woman with rectal ca s / p end ileostomy p / w ___ and anemia / / evaluate placement of NGT evaluate placement of NGT IMPRESSION : In comparison with the study of ___ , the tip of the nasogastric tube is in the mid portion of the stomach , with the side port distal to the esophagogastric junction . Otherwise , little change in the appearance of the heart and lungs except for mild increase in the degree atelectasis at the left base .

{'tokens': 'the side port', 'label': 'OBS-DP', 'start_ix': 65, 'end_ix': 67, 'relations': [['located_at', '10']]}

{'1': {'tokens': 'tip', 'label': 'OBS-DP', 'start_ix': 50, 'end_ix': 50, 'relations': [['modify', '3']]}, '2': {'tokens': 'nasogastric', 'label': 'OBS-DP', 'start_ix': 53, 'end_ix': 53, 'relations': [['modify', '3']]}, '3': {'tokens': 'tube', 'label': 'OBS-DP', 'start_ix': 54, 'end_ix': 54, 'relations': [['located_at', '6']]}, '4': {'tokens': 'mid', 'label': 'ANAT-DP', 

In [101]:
s, l = next(iter(tmp['test'].items()))
print(s)
for x in l:
    print(tmp['hash2string'][x])

FINDINGS : The lungs are clear without focal consolidation , , or edema
edema|OBS-DA
focal|OBS-DA|modify|consolidation|OBS-DA
consolidation|OBS-DA|located_at|lungs|ANAT-DP
clear|OBS-DP|lungs|ANAT-DP
edema|OBS-DA|located_at|lungs|ANAT-DP
edema|OBS-DA|lungs|ANAT-DP
clear|OBS-DP|located_at|lungs|ANAT-DP
consolidation|OBS-DA
lungs|ANAT-DP
clear|OBS-DP
focal|OBS-DA|consolidation|OBS-DA
consolidation|OBS-DA|lungs|ANAT-DP
focal|OBS-DA


In [207]:
from nltk.tokenize import sent_tokenize
import re

_ONLY_UNDERSCORES_REGEX = re.compile(r'^_+$')

def _extract_entities_and_relations_per_sentence(text, entities, max_sentences_per_entity=2, max_sentences_per_relation=1,
                                                 clean_text=True, entity_radius=3):
    sentences = sent_tokenize(text)
    ns = len(sentences)
    sentence_tokens = [s.split() for s in sentences]
    sentence_offsets = [0] * ns
    for i in range(1, ns):
        sentence_offsets[i] = sentence_offsets[i-1] + len(sentence_tokens[i-1])

    # Clean text
    if clean_text:
        n_tokens = sum([len(s) for s in sentence_tokens])
        counts = [0] * n_tokens
        for e in entities.values():
            s_idx = max(e['start_ix'] - entity_radius, 0)
            e_idx = min(e['end_ix'] + entity_radius, n_tokens-1) + 1
            counts[s_idx] += 1
            if e_idx < n_tokens:
                counts[e_idx] -= 1
        for i in range(1, n_tokens):
            counts[i] += counts[i-1]
        i = 0
        for s in range(ns):
            for t in range(len(sentence_tokens[s])):
                if counts[i] == 0 or _ONLY_UNDERSCORES_REGEX.match(sentence_tokens[s][t]):
                    sentence_tokens[s][t] = ''
                i += 1
            sentence_tokens[s] = [t for t in sentence_tokens[s] if t != '']
            sentences[s] = ' '.join(sentence_tokens[s])
            

    pairs_to_ent_rels = {}
    
    # Add entities
    e_pairs = {}
    for k, e in entities.items():
        start_sent_idx = _token_idx_to_sentence_idx(e['start_ix'], sentence_offsets)
        end_sent_idx = _token_idx_to_sentence_idx(e['end_ix'], sentence_offsets)
        p = (start_sent_idx, end_sent_idx)
        assert p[0] <= p[1]
        if p[1] - p[0] >= max_sentences_per_entity:
            continue # skip entities spanning more than max_sentences_per_entity
        e_string = f'{e["tokens"].lower()}|{e["label"]}'
        assert start_sent_idx <= end_sent_idx
        try:
            pairs_to_ent_rels[p].append(e_string)
        except KeyError:
            pairs_to_ent_rels[p] = [e_string]
        e_pairs[k] = (e_string, p)
    
    # Add relations
    for k, e in entities.items():
        try:
            ep1 = e_pairs[k]
        except KeyError:
            continue
        for r in e['relations']:
            try:
                ep2 = e_pairs[r[1]]
            except KeyError:
                continue
            start_sent_idx = min(ep1[1][0], ep2[1][0])
            end_sent_idx = max(ep1[1][1], ep2[1][1])
            p = (start_sent_idx, end_sent_idx)
            if p[1] - p[0] >= max_sentences_per_relation:
                continue # skip relations spanning more than max_sentences_per_relation
            r_string_1 = f'{ep1[0]}|{r[0]}|{ep2[0]}' # e1|rel|e2
            r_string_2 = f'{ep1[0]}|{ep2[0]}' # e1|e2
            try:
                pairs_to_ent_rels[p].append(r_string_1)
                pairs_to_ent_rels[p].append(r_string_2)
            except KeyError:
                pairs_to_ent_rels[p] = [r_string_1, r_string_2]

    # Sort pairs by length
    if len(pairs_to_ent_rels) > 1:
        pairs = list(pairs_to_ent_rels.keys())
        pairs.sort(key=lambda x: x[1]-x[0], reverse=True)
        len_last = pairs[-1][1] - pairs[-1][0]
        for i in range(len(pairs)):
            len_i  = pairs[i][1] - pairs[i][0]
            if len_i == len_last:
                break
            for j in range(i+1, len(pairs)):
                len_j = pairs[j][1] - pairs[j][0]
                if len_i > len_j and pairs[i][0] <= pairs[j][0] and pairs[i][1] >= pairs[j][1]:
                    # pairs[i] contains pairs[j] -> include all entities and relations
                    # from pairs[j] in pairs[i]
                    pairs_to_ent_rels[pairs[i]].extend(pairs_to_ent_rels[pairs[j]])

    return sentences, pairs_to_ent_rels

In [193]:
_p = next(iter(mimiccxr_graphs.items()))

In [194]:
_p

('p10/p10659857/s59206984.txt',
 {'text': 'FINAL REPORT INDICATION : ___ year old woman with post-op hypoxemia / / any consolidation ? TECHNIQUE : Chest PA and lateral COMPARISON : Chest radiograph from ___ FINDINGS : The lung volumes are normal . No focal consolidations . The cardiomediastinal contours are normal . A small right pleural effusion . The left pleural surfaces are normal . No pneumothorax . Stable thoracolumbar scoliosis . Stable left glenohumeral arthroplasty . Stable appearance of lumbar spinal fusion hardware . The left chest port catheter terminates in the upper right atrium , unchanged . IMPRESSION : No acute cardiopulmonary process .',
  'entities': {'1': {'tokens': 'lung',
    'label': 'ANAT-DP',
    'start_ix': 31,
    'end_ix': 31,
    'relations': []},
   '2': {'tokens': 'volumes',
    'label': 'ANAT-DP',
    'start_ix': 32,
    'end_ix': 32,
    'relations': [['modify', '1']]},
   '3': {'tokens': 'normal',
    'label': 'OBS-DP',
    'start_ix': 34,
    'end_ix'

In [216]:
_extract_entities_and_relations_per_sentence(
    text=_p[1]['text'],
    entities=_p[1]['entities'],
    clean_text=True,
    entity_radius=3,
)

(['',
  'FINDINGS : The lung volumes are normal .',
  'No focal consolidations .',
  'The cardiomediastinal contours are normal .',
  'A small right pleural effusion .',
  'The left pleural surfaces are normal .',
  'No pneumothorax .',
  'Stable thoracolumbar scoliosis .',
  'Stable left glenohumeral arthroplasty .',
  'Stable appearance of lumbar spinal fusion hardware .',
  'The left chest port catheter terminates in the upper right atrium , unchanged .',
  'IMPRESSION : No acute cardiopulmonary process .'],
 {(1, 1): ['lung|ANAT-DP',
   'volumes|ANAT-DP',
   'normal|OBS-DP',
   'volumes|ANAT-DP|modify|lung|ANAT-DP',
   'volumes|ANAT-DP|lung|ANAT-DP',
   'normal|OBS-DP|located_at|lung|ANAT-DP',
   'normal|OBS-DP|lung|ANAT-DP'],
  (2, 2): ['focal|OBS-DA',
   'consolidations|OBS-DA',
   'focal|OBS-DA|modify|consolidations|OBS-DA',
   'focal|OBS-DA|consolidations|OBS-DA'],
  (3, 3): ['cardiomediastinal|ANAT-DP',
   'contours|ANAT-DP',
   'normal|OBS-DP',
   'contours|ANAT-DP|modify|car